In [1]:
#load libraries
import ee
import datetime

In [2]:
#target output
target_crs = 'SR-ORG:6974'

In [3]:
#Init earth engine and load a collection
ee.Initialize()
imgs = ee.ImageCollection('MODIS/006/MOD11A2')

In [4]:
#set region of interest
#define a bounding box
bbox = (38.94993166800003, -7.127943276999929, 39.44200277300007, -6.552704129999938)
thecity = ee.Geometry.Rectangle(bbox)

In [5]:
#filer by some things and extract one image
dat = imgs.filterDate('2004-01-01','2004-01-02').select('LST_Day_1km').toList(1000)

In [6]:
#For some reason ee sees dat as a computed object
#trick it
dat = ee.Image.cat([dat.get(0)])

In [7]:
#only clipping, keep bbox in lat long format
ask1 = ee.batch.Export.image.toDrive(image = dat, description = 'ask1', fileNamePrefix = 'ask1',
                                     region = thecity.getInfo().get('coordinates'))

In [8]:
ask1.start()

In [18]:
#enforce a projection
#extent is specified in meters--
#+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs
bbox = (38.94993166800003, -7.127943276999929, 39.44200277300007, -6.552704129999938)
thecity = ee.Geometry.Rectangle(bbox)
target_crs = 'SR-ORG:6974'
tp = ee.Projection(target_crs)
err = ee.ErrorMargin(ee.Number(100.0))
thecity =thecity.transform(tp, err)

dat2 = imgs.filterDate('2004-01-01','2004-01-02').select('LST_Day_1km')
dat2 = ee.Image.cat([dat2])
dat2 = dat.reproject(tp)
ask2 = ee.batch.Export.image.toDrive(image = dat2, description = 'ask2', fileNamePrefix = 'ask2',
                                    region = thecity.getInfo().get('coordinates'))

In [19]:
ask2.start()

In [13]:
#enforce a different projection
bbox = (38.94993166800003, -7.127943276999929, 39.44200277300007, -6.552704129999938)
thecity = ee.Geometry.Rectangle(bbox)
target_crs = """PROJCS["Sinusoidal",
    GEOGCS["GCS_Undefined",
        DATUM["Undefined",
            SPHEROID["User_Defined_Spheroid",6371007.181,0.0]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Sinusoidal"],
    PARAMETER["False_Easting",0.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["Central_Meridian",0.0],
    UNIT["Meter",1.0]]"""
tp = ee.Projection(target_crs)
err = ee.ErrorMargin(ee.Number(100.0))
thecity =thecity.transform(tp, err)

dat3 = imgs.filterDate('2004-01-01','2004-01-02').select('LST_Day_1km')
dat3 = ee.Image.cat([dat3])
dat3 = dat.reproject(tp)
ask3 = ee.batch.Export.image.toDrive(image = dat3, description = 'ask3', fileNamePrefix = 'ask3',
                                     region = thecity.getInfo().get('coordinates'))

In [15]:
ask3.start()

In [16]:
#try 4: set output projection ee.batch.export

'PROJCS["Sinusoidal",\n    GEOGCS["GCS_Undefined",\n        DATUM["Undefined",\n            SPHEROID["User_Defined_Spheroid",6371007.181,0.0]],\n        PRIMEM["Greenwich",0.0],\n        UNIT["Degree",0.0174532925199433]],\n    PROJECTION["Sinusoidal"],\n    PARAMETER["False_Easting",0.0],\n    PARAMETER["False_Northing",0.0],\n    PARAMETER["Central_Meridian",0.0],\n    UNIT["Meter",1.0]]'

In [ ]:
#try 5: buffer the bounding box, and crop post hoc in R

In [20]:
thecity